In [1]:
import sys
sys.path.append("/sps/lsst/users/ebarroso/crow")
import crow
from crow.deltasigma import *

## Testing Shear implementation in deltasigma.py

In [2]:
import pyccl as ccl
hmf = ccl.halos.MassFuncTinker08(mass_def="200c")

cl_delta_sigma = ClusterDeltaSigma((12,15), (0.1,1.2), hmf, True, 4.0)
cosmo = ccl.Cosmology(
    Omega_c=0.2607,      # Cold dark matter density
    Omega_b=0.04897,     # Baryon density
    h=0.6766,            # Hubble parameter
    sigma8=0.8102,       # Matter fluctuation amplitude
    n_s=0.9665,          # Spectral index
)
cl_delta_sigma._cosmo = cosmo

cl_reduced_shear = ClusterDeltaSigma((12,15), (0.1,1.2), hmf, False, 4.0)

cl_reduced_shear._cosmo = cosmo

In [3]:
ds_val = cl_delta_sigma.delta_sigma([13], [1.2], 3.0)
print(ds_val)

gt_val = cl_reduced_shear.delta_sigma([13], [1.2], 3.0)
print(gt_val)

[1.13451921e+12]
[0.00027905]


## Values with interpolated Beta

In [4]:
gt_val = cl_reduced_shear.delta_sigma([13], [1.2], 3.0, use_beta_interp=True)
print(gt_val)
cl_reduced_shear.beta_zbin_cl_edges = [0.2, 0.4]
cl_reduced_shear.beta_interp = None
gt_val = cl_reduced_shear.delta_sigma([13], [1.2], 3.0, use_beta_interp= True)
print(gt_val)

Warning, no redshift bin for beta estimation, using integration values (0.1, 1.2) instead
[0.00027905]
[0.00045063]


## Test implementation on the recipe

In [9]:
from crow.recipes.murata_binned_spec_z_deltasigma import MurataBinnedSpecZDeltaSigmaRecipe
from crow.mass_proxy import MurataBinned
from crow.kernel import SpectroscopicRedshift
from firecrown.models.cluster import ClusterProperty
import time

pivot_mass, pivot_redshift = 14.625862906, 0.6
mass_distribution = MurataBinned(pivot_mass, pivot_redshift)
redshift_distribution = SpectroscopicRedshift()

def run_recipe(label, is_delta_sigma=False, use_beta_interp=False):
    print(f"\n{label} Starts here\n{'-'*50}")
    
    recipe = MurataBinnedSpecZDeltaSigmaRecipe(
        hmf=hmf,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
        is_delta_sigma=is_delta_sigma,
        use_beta_interp=use_beta_interp
    )
    recipe.cluster_theory._cosmo = cosmo

    t0 = time.time()
    val1 = recipe.evaluate_theory_prediction((1.1, 1.2), (1, 2), 1.0, 440, ClusterProperty.DELTASIGMA)
    t1 = time.time()
    print(f"Eval 1: {val1:.3e} | Time: {t1-t0:.4f} s")

    t2 = time.time()
    val2 = recipe.evaluate_theory_prediction((0.2, 0.31), (0.86, 1.0), 4.0, 440, ClusterProperty.DELTASIGMA)
    t3 = time.time()
    print(f"Eval 2: {val2:.3e} | Time: {t3-t2:.4f} s")

    counts = recipe.evaluate_theory_prediction_counts((0.2, 0.31), (0.86, 1.0), 440)
    print(f"Counts: {counts:.3e}")

# Run different scenarios
run_recipe("Delta Sigma", is_delta_sigma=True)
run_recipe("Tangential Shear", is_delta_sigma=False)
run_recipe("Tangential Shear with Interpolation", is_delta_sigma=False, use_beta_interp=True)



Delta Sigma Starts here
--------------------------------------------------
Eval 1: 8.205e+15 | Time: 0.2356 s
Eval 2: 1.281e+15 | Time: 0.2376 s
Counts: 2.658e+02

Tangential Shear Starts here
--------------------------------------------------
Eval 1: 2.099e+00 | Time: 2.2253 s
Eval 2: 4.338e-01 | Time: 7.6118 s
Counts: 2.658e+02

Tangential Shear with Interpolation Starts here
--------------------------------------------------
Eval 1: 2.099e+00 | Time: 0.2801 s
Eval 2: 4.338e-01 | Time: 0.3233 s
Counts: 2.658e+02
